In [ ]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch import from_numpy
from torch.utils.data import DataLoader, TensorDataset

from neural_net import Net2
from preprocessing import one_hot_encode_sequences, read_all_data

In [ ]:
from util import test_network, train_network
df = read_all_data("data/ss_out.txt",
                   "data/3U_sequences_final.txt",
                   "data/3U.models.3U.40A.seq1022_param.txt",
                   "data/3U.models.3U.00A.seq1022_param.txt")
df

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
sequences_tensor = one_hot_encode_sequences(df["sequence"], drop_first=True)
sequences_tensor

In [ ]:
seq_train, \
seq_test, \
rate_a_plus_train, \
rate_a_plus_test, \
rate_a_minus_train, rate_a_minus_test = \
    train_test_split(sequences_tensor,
                     df["log2_deg_rate_a_plus"].to_numpy(np.float32).reshape(-1, 1),
                     df["log2_deg_rate_a_minus"].to_numpy(np.float32).reshape(-1, 1),
                     train_size=0.9)

In [ ]:
a_plus_net = Net2()
a_minus_net = Net2()
a_minus_net

In [ ]:
train_loader = DataLoader(TensorDataset(from_numpy(seq_train), from_numpy(rate_a_plus_train)),
                          batch_size=4, shuffle=True)
optimizer = torch.optim.Adam(a_plus_net.parameters(), lr=0.001)
train_network(a_plus_net, train_loader, 5, nn.MSELoss(), optimizer)

In [ ]:
train_loader = DataLoader(TensorDataset(from_numpy(seq_train), from_numpy(rate_a_minus_train)),
                          batch_size=4, shuffle=True)
optimizer = torch.optim.Adam(a_minus_net.parameters(), lr=0.001)
train_network(a_minus_net, train_loader, 5, nn.MSELoss(), optimizer)

In [ ]:
test_loader = DataLoader(TensorDataset(from_numpy(seq_test), from_numpy(rate_a_plus_test)),
                         batch_size=4, shuffle=False)
a_plus_mse = test_network(a_plus_net, test_loader)
a_plus_mse

In [ ]:
test_loader = DataLoader(TensorDataset(from_numpy(seq_test), from_numpy(rate_a_minus_test)),
                         batch_size=4, shuffle=False)
a_minus_mse = test_network(a_minus_net, test_loader)
a_minus_mse
